<a href="https://colab.research.google.com/github/Skuerjuan/tp2-chona/blob/main/maierowicz_atanasoff_gallo_ABD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 Seminario IA
# Maierowicz, Atanasoff, Gallo
---
## ¿Deberían aceptar darle el préstamo?
### (Esta vez con redes neuronales)

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
# Preprocesamiento

data = "https://raw.githubusercontent.com/Skuerjuan/tp1_chona/refs/heads/main/loan_data.csv"
df = pd.read_csv(data)

categorical = ["person_gender", "person_education", "person_home_ownership", "loan_intent", "previous_loan_defaults_on_file"]

ct = ColumnTransformer(transformers=[("cat", OneHotEncoder(), categorical)], remainder="passthrough")

x = ct.fit_transform(df)

feature_names = ct.get_feature_names_out()

df = pd.DataFrame(x, columns=feature_names)
df_x = df.drop(columns="remainder__loan_status")
df_y = df["remainder__loan_status"]

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [3]:
# deberia haber un hyperparam de cuantas layers hay en el medio y otro de cuanto es el n
# otro más que sea el activation sería una buena idea (?)

model = keras.Sequential([
    layers.Input(shape=(x_train.shape[1],)),
    layers.Dense(15, activation="relu"),
    layers.Dense(2, activation="softmax")
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 15)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            32 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 452 (1.77 KB)

 Trainable params: 452 (1.77 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 536us/step - accuracy: 0.7396 - loss: 48.7548 
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 540us/step - accuracy: 0.7279 - loss: 12.5792
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 520us/step - accuracy: 0.7355 - loss: 12.8782
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 515us/step - accuracy: 0.7426 - loss: 11.5084
Epoch 5/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 537us/step - accuracy: 0.7468 - loss: 10.4156
Epoch 6/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 529us/step - accuracy: 0.7547 - loss: 10.4960
Epoch 7/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 513us/step - accuracy: 0.7576 - loss: 13.1578
Epoch 8/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 512us/step - accuracy: 0.7572 - loss: 9.1414
Epoch 9/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 519us/step - accuracy: 0.7666 - loss: 8.8529
Epoch 10/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 539us/step - accuracy: 0.7707 - loss: 8.6370


In [5]:
score = model.evaluate(x_test, y_test, batch_size=64, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

y_pred_probs = model.predict(x_test)
y_pred_labels = np.argmax(y_pred_probs, axis=1)

print(f'Precision: {precision_score(y_test, y_pred_labels)}\n')
print(f'Recall: {recall_score(y_test, y_pred_labels)}\n')

Test loss: 22.67115020751953
Test accuracy: 0.7777777910232544
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 370us/step
Precision: 0.9166666666666666

Recall: 0.005472636815920398



In [32]:
# evaluar el mejor set de hyperparams que quede al final
# comparar con el tp anterior
from sklearn.model_selection import KFold

EPOCHS = 10
N_SPLITS = 5

def model_with_params(layers_num: int, neurons_per_layer: int, activation: str = "relu"):
  model = keras.Sequential()
  model.add(layers.Input(shape=(x_train.shape[1],)))
  for i in range(layers_num):
    model.add(layers.Dense(neurons_per_layer, activation=activation))
  model.add(layers.Dense(2, activation="softmax"))
  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  return model

# Sacado de acá: https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md
def kfold_validation(model_opts, kfold: KFold):
  fold_no = 1
  acc_per_fold = []
  loss_per_fold = []
  _df_x = df_x.to_numpy()
  _df_y = df_y.to_numpy()
  models = []
  for train, test in kfold.split(df_x, df_y):
#    model = model_with_params(layers_num=model_opts[0], neurons_per_layer=model_opts[1], activation=model_opts[2])
    model = model_with_params(1, 1)
    print(f'Training for fold {fold_no} ...')
    model.fit(_df_x[train], _df_y[train], epochs=EPOCHS, verbose=1)
    models.append(model)
    print(f'Evaluating...')
    score = model.evaluate(_df_x[test], _df_y[test], verbose=1)
    print(f'Done...')
    acc_per_fold.append(score[1])
    loss_per_fold.append(score[0])
    fold_no += 1
  return {
      "accuracy": np.mean(acc_per_fold),
      "loss": np.mean(loss_per_fold),
      "models": models
  }



In [33]:
# Entrenandolos

modelos = [
  # Probamos uno grande
  [5, 15, "relu"],
  # Probamos uno chico
  [2, 5, "relu"],
  # Los mismos pero con otra función de activación
  [5, 15, "sigmoid"],
  [2, 5, "softplus"],
]

kfold = KFold(n_splits=N_SPLITS, shuffle=True)
# Guardamos resultados
kfold_results = []

# Entrenamiento guardando los resultados
# No termina más
for model_params in modelos:
  kfold_results.append(kfold_validation(model_params, kfold))

Training for fold 1 ...
Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 525us/step - accuracy: 0.7776 - loss: 1.5449
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 521us/step - accuracy: 0.7776 - loss: 0.5311
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 518us/step - accuracy: 0.7776 - loss: 0.5301
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 525us/step - accuracy: 0.7776 - loss: 0.5300
Epoch 5/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 529us/step - accuracy: 0.7776 - loss: 0.5300
Epoch 6/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 488us/step - accuracy: 0.7776 - loss: 0.5300
Epoch 7/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 494us/step - accuracy: 0.7776 - loss: 0.5300
Epoch 8/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 517us/step - accuracy: 0.7776 - loss: 0.5299
Epoch 9/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 492us/step - accuracy: 0.7776 - loss: 0.5300
Epoch 10/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 509us/step - accuracy: 0.7776 - loss: 0.5300
Evaluating...
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.7784

In [34]:
max_accuracy = 0
best_model = 0
current_model = 0
print("-"*32)
for result in kfold_results:
    print(f"Modelo {current_model}")
    print(f"accuracy: {result['accuracy']}")
    print(f"loss: {result['loss']}")
    print("-"*32)
    
    if result['accuracy'] > max_accuracy:
        best_model = current_model
        max_accuracy = result['accuracy']
    current_model += 1

print(f"Mejor modelo en base de accuracy: {best_model}")

--------------------------------
Modelo 0
accuracy: 0.7777777910232544
loss: 0.5297071814537049
--------------------------------
Modelo 1
accuracy: 0.7777777671813965
loss: 0.5176205217838288
--------------------------------
Modelo 2
accuracy: 0.7777777791023255
loss: 0.5164701163768768
--------------------------------
Modelo 3
accuracy: 0.7857111096382141
loss: 0.5098878979682923
--------------------------------
Mejor modelo en base de accuracy: 3


In [35]:
# Resultados del mejor modelo y del tp 1
y_preds = kfold_results[best_model]["models"][0].predict(x_test, verbose=0, batch_size=64)
y_pred_labels = np.argmax(y_pred_probs, axis=1)

print("Mejor modelo")
print(f'Precision: {precision_score(y_test, y_pred_labels)}')
print(f'Recall: {recall_score(y_test, y_pred_labels)}')
score = model.evaluate(x_test, y_test, batch_size=64, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

print("-"*32)
print("Modelo del TP1")

print("Precision del TP1: 0.8997555012224939")
print("Recall del TP1: 0.7323383084577114")
print("Accuracy: 0.922")

Mejor modelo
Precision: 0.9166666666666666
Recall: 0.005472636815920398
Test loss: 22.67115020751953
Test accuracy: 0.7777777910232544
--------------------------------
Modelo del TP1
Precision del TP1: 0.8997555012224939
Recall del TP1: 0.7323383084577114
Accuracy: 0.922


In [ ]:
# Conclusión:
# La diferencia que se puede apreciar entre el mejor modelo de este tp (M2) y el modelo del tp1 (M1) se puede asumir
# que es causada debido a las pocas epochs, 10, que tuvo M2 para ser entrenado. Esta poca cantidad de epochs fue una
# decisión tomando en cuenta el tiempo y coste computacional de añadir más epochs, ya que sin duda, asumiendo tiempo
# infinito, M2 hubiera superado a M1. También se podría estimar que M1 tal vez sufra de overfitting, pero eso es
# algo que quedará incierto, ya que probablemente se podría seguir entrenando.

# En pocas palabras, si M2 hubiera tenido mayor entrenamiento, se podría asumir que hubiera superado a M1 en la
# mayoría, por no decir todas, de las métricas evaluadas.